### In this demo, you'll see different modes of AES block cipher and their properties.

In [1]:
from Crypto.Cipher import AES # NOTE: you may need to install PyCrypto package
from Crypto.Random import get_random_bytes
from Crypto.Util import Counter
import random

In [2]:
def pad(data, length=16):
    """pads data to nearest multiple of length (default 16 bytes = 128 bits)"""
    return data + "-" * (16 - (len(data) % length))

def hexify(data):
    # Source: https://stackoverflow.com/questions/12214801/print-a-string-as-hex-bytes
    return ":".join("{:02x}".format(c) for c in data)
    

plaintext = "The world wonders"
            #----------------=
    
# Some modes of operation require padding to nearest 128 bits (16 bytes)
plaintext_padded = pad(plaintext)

key = get_random_bytes(16)

In [3]:
def blockPrint(data):
    index = 0
    hexdata = ":".join("{:02x}".format(c) for c in data)
    length = len(hexdata)
    while index < length:
        print(hexdata[index:min(index + 2*16 + 15, length)])
        index += 3*16

def xorBlockPrint(c1, c2):
    index = 0
    xor = ":".join(["{:02x}".format(c1[c] ^ c2[c]) for c in range(len(c1))]).replace("00", "--")
    length = len(xor)
    while index < length:
        print(xor[index:min(index + 2*16 + 15, length)])
        index += 3*16

In [4]:
def encryptRandom(algorithm, plaintext1, plaintext2):
    """Encrypts either plaintext at random and returns the ciphertext.
       Used when playing IND-CPA games."""
    return algorithm.encrypt(random.choice([plaintext1, plaintext2]))

## IND-CPA with deterministic algorithm: AES-ECB

### Encryption

In [5]:
AES_ECB = AES.new(key, AES.MODE_ECB)

print("Encryption of: ", plaintext_padded)
blockPrint(AES_ECB.encrypt(plaintext_padded))

Encryption of:  The world wonders---------------
b8:b8:2d:69:1f:da:2c:03:62:5b:1b:c4:4a:13:f8:6d
83:8d:9d:65:24:ed:2d:fa:4b:6a:76:b0:fb:94:e9:32


#### If you reencrypt the same plaintext, the ciphertext is identical

In [6]:
print("Encryption of: ", plaintext_padded)
blockPrint(AES_ECB.encrypt(plaintext_padded))

Encryption of:  The world wonders---------------
b8:b8:2d:69:1f:da:2c:03:62:5b:1b:c4:4a:13:f8:6d
83:8d:9d:65:24:ed:2d:fa:4b:6a:76:b0:fb:94:e9:32


### Decryption

In [7]:
ciphertext = AES_ECB.encrypt(plaintext_padded)
print("Decryption of: ")
blockPrint(ciphertext)
print(AES_ECB.decrypt(ciphertext))

Decryption of: 
b8:b8:2d:69:1f:da:2c:03:62:5b:1b:c4:4a:13:f8:6d
83:8d:9d:65:24:ed:2d:fa:4b:6a:76:b0:fb:94:e9:32
b'The world wonders---------------'


### For an attacker, winning the IND-CPA game against AES-ECB is easy

In [8]:
"""Wins the IND-CPA game against AES-ECB as an attacker."""

# Main game: provide two plaintexts, cipher selects one
plaintext1 = pad("This is a plaintext")
plaintext2 = pad("A completely different one")
ciphertext = encryptRandom(AES_ECB, plaintext1, plaintext2)
print("Ciphertext returned by the oracle:")
blockPrint(ciphertext)

# Attacker can obtain additional encryptions of plaintexts.
ciphertext1 = AES_ECB.encrypt(plaintext1)
ciphertext2 = AES_ECB.encrypt(plaintext2)
print("\nEncryption of plaintext 1:")
blockPrint(ciphertext1)
print("\nEncryption of plaintext 2:")
blockPrint(ciphertext2)

# Make a guess!
if ciphertext == ciphertext1:
    print("\nOracle's ciphertext is encryption of plaintext 1")
else:
    print("\nOracle's ciphertext is encryption of plaintext 2")

Ciphertext returned by the oracle:
76:37:ff:5a:44:e5:03:92:53:94:99:67:e4:02:02:0a
8b:98:7f:62:0e:3e:1e:9e:77:a9:9d:d2:17:fc:77:22

Encryption of plaintext 1:
39:88:3c:39:38:3c:e0:d7:64:f0:b8:e1:fa:fe:23:58
cb:ad:14:c9:42:e0:66:00:51:e5:4d:b6:f7:20:e4:5f

Encryption of plaintext 2:
76:37:ff:5a:44:e5:03:92:53:94:99:67:e4:02:02:0a
8b:98:7f:62:0e:3e:1e:9e:77:a9:9d:d2:17:fc:77:22

Oracle's ciphertext is encryption of plaintext 2


## IND-CPA with nondeterministic algorithm: AES-CTR

### Encryption

In [9]:
print("Encryption of: ", plaintext) #Notice no padding is required!
IV = get_random_bytes(8)
print("IV used: ", hexify(IV))
blockPrint(AES.new(key, AES.MODE_CTR, counter=Counter.new(64, prefix=IV)).encrypt(plaintext))

Encryption of:  The world wonders
IV used:  3c:85:98:fb:ea:7e:90:a4
44:81:16:c7:93:f9:0f:b6:af:2e:d7:5e:ce:bb:07:03
68


#### If you reencrypt the same plaintext with a different IV, the ciphertext is different. This makes an encryption scheme IND-CPA since no correlation can be established between multiple encryptions of similar or same plaintext.

In [10]:
print("Encryption of: ", plaintext)
IV = get_random_bytes(8)
print("IV used: ", hexify(IV))
blockPrint(AES.new(key, AES.MODE_CTR, counter=Counter.new(64, prefix=IV)).encrypt(plaintext))

Encryption of:  The world wonders
IV used:  2f:bd:20:98:1a:db:b7:b2
51:79:c5:3f:a7:88:70:dd:d0:98:32:9d:6b:ee:5f:58
08


#### However, if you reencrypt the same plaintext with the same IV, the ciphertext is identical

In [11]:
print("Two encryptions of: ", plaintext)
IV = get_random_bytes(8)
print("IV used: ", hexify(IV))
print("First encryption:")
blockPrint(AES.new(key, AES.MODE_CTR, counter=Counter.new(64, prefix=IV)).encrypt(plaintext))
print("\nSecond encryption:")
blockPrint(AES.new(key, AES.MODE_CTR, counter=Counter.new(64, prefix=IV)).encrypt(plaintext))

Two encryptions of:  The world wonders
IV used:  79:7f:48:0f:1e:f5:c6:6d
First encryption:
e6:88:f3:42:81:9e:43:3a:7c:b4:29:f9:5b:91:65:6c
c5

Second encryption:
e6:88:f3:42:81:9e:43:3a:7c:b4:29:f9:5b:91:65:6c
c5


### Again, wining the IND-CPA game against AES-CTR with IV reuse is easy

In [ ]:
"""Wins the IND-CPA game against AES-CTR with IV reuse as an attacker."""

#Generate IV to reuse
IV = get_random_bytes(8) 

# Main game: provide two plaintexts, cipher selects one
plaintext1 = "A good plaintext to encrypt"
plaintext2 = "This is another one for you"
ciphertext = encryptRandom(AES.new(key, AES.MODE_CTR, counter=Counter.new(64, prefix=IV)), plaintext1, plaintext2)
print("Ciphertext returned by the oracle:")
blockPrint(ciphertext)

# Attacker can obtain additional encryptions of plaintexts.
ciphertext1 = AES.new(key, AES.MODE_CTR, counter=Counter.new(64, prefix=IV)).encrypt(plaintext1)
ciphertext2 = AES.new(key, AES.MODE_CTR, counter=Counter.new(64, prefix=IV)).encrypt(plaintext2)
print("\nEncryption of plaintext 1:")
blockPrint(ciphertext1)
print("\nEncryption of plaintext 2:")
blockPrint(ciphertext2)

#Make a guess!
if ciphertext == ciphertext1:
    print("\nOracle's ciphertext is encryption of plaintext 1")
else:
    print("\nOracle's ciphertext is encryption of plaintext 2")

# IV compromise
If you encrypt a same plaintext with the same IV, two ciphertexts will be identical regardless of the mode of operation. If you encrypt two different plaintexts with the same IV, though, different modes of operation have differing levels of failure. In this section, you will analyze what happens when IV is reused in various AES modes of operation.

## IV reuse in synchronous stream ciphers [AES-CTR and AES-OFB]
Synchronous stream ciphers generate a psuedorandom bit stream that is combined (usually using xor) with plaintext, emulating One Time Pad (OTP). A Synchronous stream cipher's psuedorandom bit stream is dependent only on the key and IV: Plaintext does not affect the bitstream.

Since the bitstream is only dependent on the key and IV, IV reuse creates an OTP reuse scenario acoss the entire ciphertext. 

In [ ]:
# Generate IV to reuse
IV = get_random_bytes(16)

# Note one byte of the plaintexts differ in the second block.
plaintext1 = "The plaintext is designed to occupy three blocks"
             #----------------================---------------- Exactly three blocks
plaintext2 = "The plaintext is designed To occupy three blocks"

# OFB mode of operation
ciphertext1 = AES.new(key, AES.MODE_OFB, IV=IV).encrypt(plaintext1)
ciphertext2 = AES.new(key, AES.MODE_OFB, IV=IV).encrypt(plaintext2)
print("\nEncryption of plaintext 1:")
blockPrint(ciphertext1)
print("\nEncryption of plaintext 2:")
blockPrint(ciphertext2)

print("\nxor of two plaintexts:")
xorBlockPrint([ord(c) for c in plaintext1], [ord(c) for c in plaintext2])

print("\nxor of two ciphertexts:")
xorBlockPrint(ciphertext1, ciphertext2)

You can tell the two plaintexts are identical except for one byte by only looking at the ciphertexts.

## IV reuse in AES-CFB
AES-CFB uses cipher feedback, which incorporates previous block of plaintext to the input of next block cipher. In case of IV reuse, all ciphertext blocks up to and including the block containing the first differing bit is subject to OTP reuse scenario.

In [ ]:
# Generate IV to reuse
IV = get_random_bytes(16)

# Note one byte of the plaintexts differ in the second block.
plaintext1 = "the plaintext is designed to occupy three blocks"
             #----------------================---------------- Exactly three blocks
plaintext2 = "the plaintext is designed To occupy three blocks"

# CFB mode of operation
ciphertext1 = AES.new(key, AES.MODE_CFB, IV=IV, segment_size=128).encrypt(plaintext1)
ciphertext2 = AES.new(key, AES.MODE_CFB, IV=IV, segment_size=128).encrypt(plaintext2)
print("\nEncryption of plaintext 1:")
blockPrint(ciphertext1)
print("\nEncryption of plaintext 2:")
blockPrint(ciphertext2)

print("\nxor of two plaintexts:")
xorBlockPrint([ord(c) for c in plaintext1], [ord(c) for c in plaintext2])

print("\nxor of two ciphertexts:")
xorBlockPrint(ciphertext1, ciphertext2)

## IV reuse in AES-CBC
AES-CBC inporporates plaintext block to the input of the block cipher. In case of IV reuse, all ciphertext blocks up to but not including the block containing the first differing bit is subject to OTP reuse scenario.

In [ ]:
# Generate IV to reuse
IV = get_random_bytes(16)

# Note one byte of the plaintexts differ in the second block.
plaintext1 = "The plaintext is designed to occupy three blocks"
             #----------------================---------------- Exactly three blocks
plaintext2 = "The plaintext is designed To occupy three blocks"

# CBC mode of operation
ciphertext1 = AES.new(key, AES.MODE_CBC, IV=IV).encrypt(plaintext1)
ciphertext2 = AES.new(key, AES.MODE_CBC, IV=IV).encrypt(plaintext2)
print("\nEncryption of plaintext 1:")
blockPrint(ciphertext1)
print("\nEncryption of plaintext 2:")
blockPrint(ciphertext2)

print("\nxor of two plaintexts:")
xorBlockPrint([ord(c) for c in plaintext1], [ord(c) for c in plaintext2])

print("\nXor of two ciphertexts:")
xorBlockPrint(ciphertext1, ciphertext2)

## Predictable IV in AES-CBC causes it to be IND-CPA insecure
AES-CBC inporporates plaintext block to the input of the block cipher. This means if the IV is known in advance, the attacker can choose a plaintext that forces a specific input into the block cipher.

In [ ]:
"""Wins the IND-CPA game against AES-CBC with predictable IV as an attacker."""

# Generate two IVs known in advance
IV1 = get_random_bytes(16)
IV2 = get_random_bytes(16)
IV3 = get_random_bytes(16)

# Note one byte of the plaintexts differ in the second block.
# Encode with utf-8 as we will be doing some xor black magic
plaintext1 = "Plain Plaintext.".encode('utf-8')
             #---------------- Exactly one block
plaintext2 = "Spicy Plaintext.".encode('utf-8')

ciphertext = encryptRandom(AES.new(key, AES.MODE_CBC, IV=IV1), plaintext1, plaintext2)
print("Ciphertext returned by the oracle:")
blockPrint(ciphertext)

forged1 = bytes([plaintext1[i] ^ IV1[i] ^ IV2[i] for i in range(len(plaintext1))])
forged2 = bytes([plaintext2[i] ^ IV1[i] ^ IV3[i] for i in range(len(plaintext1))])
# Attacker can obtain additional encryptions of plaintexts.
ciphertext1 = AES.new(key, AES.MODE_CBC, IV=IV2).encrypt(forged1)
ciphertext2 = AES.new(key, AES.MODE_CBC, IV=IV3).encrypt(forged2)

print("\nEncryption of plaintext 1:")
blockPrint(ciphertext1)
print("\nEncryption of plaintext 2:")
blockPrint(ciphertext2)

# Make a guess!
if ciphertext == ciphertext1:
    print("\nOracle's ciphertext is encryption of plaintext 1")
else:
    print("\nOracle's ciphertext is encryption of plaintext 2")

## Conclusion: 
Nondeterministic algorithms rely on IV to provide randomness. Reusing IV or letting attacker predict IV may compromise cipher integrity in various levels depending on the mode of operation.